In [1]:
import tkinter as tk
from tkinter import font
from PIL import Image, ImageTk
import math

class GameTreeNode:
    """Represents a node in the game tree."""
    def __init__(self, parent, current_number, move, player_type):
        self.parent = parent  # Parent node
        self.current_number = current_number  # Current number in the game
        self.move = move  # Multiplier used to reach this state
        self.player_type = player_type  # "player" or "computer"
        self.children = []  # Child nodes

    def add_child(self, child):
        """Add a child node to this node."""
        self.children.append(child)

class GameTree:
    """Represents the game tree."""
    def __init__(self):
        self.root = None  # Root node of the tree

    def insert_node(self, parent, current_number, move, player_type):
        """Insert a node into the game tree."""
        new_node = GameTreeNode(parent, current_number, move, player_type)
        if parent is None:
            self.root = new_node  # Set as root if no parent
        else:
            parent.add_child(new_node)  # Add as child to the parent
        return new_node

class Game:
    def __init__(self, master):
        self.master = master
        self.master.title("Multiplication Game")
        self.master.geometry("700x725")
        self.master.configure(bg="#561C24")  # Set the background color

        # Initialize game tree
        self.game_tree = GameTree()
        self.current_node = None  # To track the current node in the game tree

        # Load win, lose, and draw images
        try:
            self.win_image = ImageTk.PhotoImage(Image.open('C://Users//anugr//Downloads//Win.jpg'))
        except Exception as e:
            print(f"Error loading win image: {e}. Using text feedback.")
            self.win_image = None

        try:
            self.lose_image = ImageTk.PhotoImage(Image.open('C://Users//anugr//Downloads//Lose.jpg'))
        except Exception as e:
            print(f"Error loading lose image: {e}. Using text feedback.")
            self.lose_image = None

        try:
            self.draw_image = ImageTk.PhotoImage(Image.open('C://Users//anugr//Downloads//Draw.jpg'))
        except Exception as e:
            print(f"Error loading draw image: {e}. Using text feedback.")
            self.draw_image = None

        self.starting_number = None
        self.current_number = None
        self.player_score = 0
        self.computer_score = 0
        self.first_player = None  # To store who plays first
        self.move_history = []  # List to store the move history

        # Custom font
        custom_font = font.Font(family="Helvetica", size=12, weight="bold")

        # Configure grid layout
        self.master.grid_columnconfigure(0, weight=1)
        self.master.grid_columnconfigure(1, weight=1)
        self.master.grid_columnconfigure(2, weight=1)
        for i in range(10):  # 10 rows to accommodate new UI elements
            self.master.grid_rowconfigure(i, weight=1)

        # Label for starting number input
        self.label = tk.Label(master, text="Choose a starting number (8-18):", bg="#E8D8C4", fg="#561C24", font=custom_font)
        self.label.grid(row=0, column=0, columnspan=3, pady=(20, 10))

        # Entry for starting number
        self.entry = tk.Entry(master, font=custom_font, bg="#ffffff", fg="#1e1e2f", borderwidth=2, relief="flat")
        self.entry.grid(row=1, column=0, columnspan=3, pady=(0, 10))

        # Label for choosing who plays first
        self.first_player_label = tk.Label(master, text="Who should play first?", bg="#E8D8C4", fg="#561C24", font=custom_font)
        self.first_player_label.grid(row=2, column=0, columnspan=3, pady=(10, 5))

        # Buttons to choose who plays first
        self.player_first_button = tk.Button(master, text="Player First", bg="#A0522D", fg="white", font=custom_font, 
                                             command=lambda: self.set_first_player("player"), relief="flat", bd=0, 
                                             activebackground="#6aa8ff", activeforeground="white")
        self.player_first_button.grid(row=3, column=0, padx=10, pady=(0, 10))

        self.computer_first_button = tk.Button(master, text="Computer First", bg="#A0522D", fg="white", font=custom_font, 
                                              command=lambda: self.set_first_player("computer"), relief="flat", bd=0, 
                                              activebackground="#6aa8ff", activeforeground="white")
        self.computer_first_button.grid(row=3, column=2, padx=10, pady=(0, 10))

        # Button to start the game
        self.start_button = tk.Button(master, text="Start Game", bg="#50c878", fg="white", font=custom_font, 
                                     command=self.start_game, relief="flat", bd=0, activebackground="#70e89e", activeforeground="white")
        self.start_button.grid(row=4, column=0, columnspan=3, pady=(10, 20))

        # Label for multiplier instructions
        self.multiplier_label = tk.Label(master, text="", bg="white", fg="#561C24", font=custom_font)
        self.multiplier_label.grid(row=5, column=0, columnspan=3, pady=(0, 10))

        # Multiplier buttons
        self.multiplier_buttons = []
        multipliers = [2, 3, 4]
        for i, multiplier in enumerate(multipliers):
            button = tk.Button(master, text=f"×{multiplier}", bg="#A0522D", fg="white", font=custom_font, 
                              command=lambda x=multiplier: self.player_turn(x), relief="flat", bd=0, 
                              activebackground="#ff8f81", activeforeground="white")
            button.grid(row=6, column=i, padx=10, pady=(0, 20))
            self.multiplier_buttons.append(button)

        # Label for results and current number
        self.result_label = tk.Label(master, text="", bg="white", fg="#561C24", font=custom_font, wraplength=350)
        self.result_label.grid(row=7, column=0, columnspan=3, pady=(0, 10))

        # Label for scores
        self.score_label = tk.Label(master, text="", bg="white", fg="#561C24", font=custom_font)
        self.score_label.grid(row=8, column=0, columnspan=3, pady=(0, 10))

        # Label for move history
        self.history_label = tk.Label(master, text="Move History:", bg="white", fg="#561C24", font=custom_font)
        self.history_label.grid(row=9, column=0, columnspan=3, pady=(0, 10))

        # Text widget to display move history
        self.history_text = tk.Text(master, height=5, width=50, bg="white", fg="#561C24", font=custom_font, wrap=tk.WORD)
        self.history_text.grid(row=10, column=0, columnspan=3, pady=(0, 10))

        # Button to reset the game
        self.reset_button = tk.Button(master, text="Reset", bg="#ff4757", fg="white", font=custom_font, 
                                     command=self.reset_game, relief="flat", bd=0, activebackground="#ff6b81", activeforeground="white")
        self.reset_button.grid(row=11, column=0, columnspan=3, pady=(0, 20))
        self.reset_button.config(state=tk.DISABLED)

    def set_first_player(self, player):
        """Set who plays first (player or computer)."""
        self.first_player = player
        self.player_first_button.config(state=tk.DISABLED)
        self.computer_first_button.config(state=tk.DISABLED)
        self.start_button.config(state=tk.NORMAL)

    def start_game(self):
        """Start the game with the chosen starting number and first player."""
        try:
            self.starting_number = int(self.entry.get())
            if 8 <= self.starting_number <= 18:
                self.current_number = self.starting_number
                self.player_score = 0
                self.computer_score = 0
                self.move_history = []  # Reset move history
                self.update_score()
                self.result_label.config(text=f"Current number: {self.current_number}")
                self.multiplier_label.config(text="Choose a multiplier:")
                self.reset_button.config(state=tk.NORMAL)
                for button in self.multiplier_buttons:
                    button.config(state=tk.NORMAL)

                # Initialize game tree with the starting number
                self.current_node = self.game_tree.insert_node(None, self.current_number, None, None)

                # Start the game based on who plays first
                if self.first_player == "computer":
                    self.master.after(1000, self.computer_turn)  # Delay computer's turn by 1 second
            else:
                self.result_label.config(text="Please choose a number between 8 and 18.")
        except ValueError:
            self.result_label.config(text="Invalid input. Please enter a valid integer.")

    def player_turn(self, multiplier):
        """Handle the player's turn."""
        if self.current_number is not None:
            self.current_number *= multiplier
            self.result_label.config(text=f"You multiplied by {multiplier}. Current number: {self.current_number}")
            self.update_scores("player")
            self.check_number()

            # Update move history
            self.move_history.append(f"Player ×{multiplier} → {self.current_number}")
            self.update_history()

            # Update game tree
            self.current_node = self.game_tree.insert_node(self.current_node, self.current_number, multiplier, "player")

            if self.current_number < 1200:
                self.master.after(1000, self.computer_turn)  # Delay computer's turn by 1 second

    def computer_turn(self):
        """Handle the computer's turn using Minimax with Alpha-Beta Pruning."""
        if self.current_number < 1200:
            best_score = -math.inf
            best_multiplier = 2
            for multiplier in [2, 3, 4]:
                new_number = self.current_number * multiplier
                score = self.minimax(new_number, 3, -math.inf, math.inf, False)
                if score > best_score:
                    best_score = score
                    best_multiplier = multiplier
            self.current_number *= best_multiplier
            self.result_label.config(text=f"Computer multiplied by {best_multiplier}. Current number: {self.current_number}")
            self.update_scores("computer")
            self.check_number()

            # Update move history
            self.move_history.append(f"Computer ×{best_multiplier} → {self.current_number}")
            self.update_history()

            # Update game tree
            self.current_node = self.game_tree.insert_node(self.current_node, self.current_number, best_multiplier, "computer")

    def minimax(self, number, depth, alpha, beta, maximizing_player):
        """Minimax algorithm with Alpha-Beta Pruning."""
        if depth == 0 or number >= 1200:
            return self.evaluate(number)

        if maximizing_player:
            max_eval = -math.inf
            for multiplier in [2, 3, 4]:
                new_number = number * multiplier
                eval = self.minimax(new_number, depth - 1, alpha, beta, False)
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return max_eval
        else:
            min_eval = math.inf
            for multiplier in [2, 3, 4]:
                new_number = number * multiplier
                eval = self.minimax(new_number, depth - 1, alpha, beta, True)
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return min_eval

    def evaluate(self, number):
        """Evaluate the current number for scoring using a heuristic."""
        if number >= 1200:
            return 0  # Game over, no further moves

        # Heuristic 1: Parity (odd/even)
        parity_score = 1 if number % 2 == 1 else -1

        # Heuristic 2: Proximity to 1200 (closer is better)
        proximity_score = (1200 - number) / 1200  # Normalized to [0, 1]

        # Combine the heuristics with weights
        weight_parity = 0.6
        weight_proximity = 0.4

        # Final heuristic score
        heuristic_score = (weight_parity * parity_score) + (weight_proximity * proximity_score)

        return heuristic_score

    def update_scores(self, player_type):
        """Update scores based on the resulting number."""
        if self.current_number % 2 == 0:
            if player_type == "player":
                self.computer_score -= 1
                points_info = f"Computer loses 1 point. (Number: {self.current_number} is even)"
            else:
                self.player_score -= 1
                points_info = f"You lose 1 point. (Number: {self.current_number} is even)"
        else:
            if player_type == "player":
                self.player_score += 1
                points_info = f"You gain 1 point. (Number: {self.current_number} is odd)"
            else:
                self.computer_score += 1
                points_info = f"Computer gains 1 point. (Number: {self.current_number} is odd)"

        self.result_label.config(text=f"{self.result_label.cget('text')}\n{points_info}")
        self.update_score()

    def check_number(self):
        """Check if the game has ended."""
        if self.current_number >= 1200:
            self.end_game()

    def update_score(self):
        """Update the score display."""
        self.score_label.config(text=f"Your Score: {self.player_score} | Computer Score: {self.computer_score}")

    def update_history(self):
        """Update the move history display."""
        self.history_text.delete(1.0, tk.END)
        for move in self.move_history:
            self.history_text.insert(tk.END, move + "\n")

    def end_game(self):
        """End the game and display the result."""
        for button in self.multiplier_buttons:
            button.config(state=tk.DISABLED)

        if self.player_score > self.computer_score:
            result_text = "Game Over! You win!"
            if self.win_image:
                self.result_label.config(image=self.win_image, compound="top")
        elif self.computer_score > self.player_score:
            result_text = "Game Over! Computer wins!"
            if self.lose_image:
                self.result_label.config(image=self.lose_image, compound="top")
        else:
            result_text = "Game Over! It's a draw!"
            if self.draw_image:
                self.result_label.config(image=self.draw_image, compound="top")
            else:
                self.result_label.config(image="")

        self.result_label.config(text=f"{result_text}\nFinal Number: {self.current_number}\nFinal Scores - You: {self.player_score}, Computer: {self.computer_score}")

    def reset_game(self):
        """Reset the game to its initial state."""
        self.starting_number = None
        self.current_number = None
        self.player_score = 0
        self.computer_score = 0
        self.first_player = None
        self.move_history = []  # Clear move history
        self.result_label.config(text="", image="")
        self.score_label.config(text="")
        self.multiplier_label.config(text="")
        self.history_text.delete(1.0, tk.END)  # Clear history text
        self.entry.delete(0, tk.END)
        self.reset_button.config(state=tk.DISABLED)
        self.player_first_button.config(state=tk.NORMAL)
        self.computer_first_button.config(state=tk.NORMAL)
        self.start_button.config(state=tk.DISABLED)
        for button in self.multiplier_buttons:
            button.config(state=tk.NORMAL)

        # Reset game tree
        self.game_tree = GameTree()
        self.current_node = None

# Create the main window
root = tk.Tk()
game = Game(root)
root.mainloop()